In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math as mt
import seaborn as sns
from tqdm import tqdm
import pandas as pd
from hangul_utils import split_syllables, join_jamos
from tqdm.auto import tqdm
import re
#from eunjeon import Mecab
from konlpy.tag import Mecab
from hanspell import spell_checker
from khaiii import KhaiiiApi

In [179]:
con_dict = [
    
    ['ㅏㅣ','ㅐ'], ['ㅑㅣ','ㅒ'], ['ㅓㅣ','ㅔ'],
    ['ㅕㅣ','ㅖ'], ['ㅗㅣ','ㅚ'], ['ㅗㅐ','ㅙ'],
    ['ㅜㅓ','ㅝ'], ['ㅜㅔ','ㅞ'], ['ㅡㅣ','ㅢ'],
    ['ㅣㅏ','ㅑ'], ['ㅣㅓ','ㅕ'], ['ㅣㅗ','ㅛ'],
    ['ㅣㅜ','ㅠ'], ['ㅡㅓ','ㅓ'], ['ㅗㅏ','ㅘ']
    
]

jongsung_list = [ 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']


#lis_beta = ['EP+EF', 'VCP+EF', 'B+EF', 'B+EP+EF', 'B+VCP+EF', 'EF','EP']

lis_beta = ['EP+EF', 'EF', 'B+EF', 'B+EP+EF']

#어말을 처리해 주기 위한 것으로, 나중에 EC등이 필요해 진다면 이 부분에 EC 등을 집어넣어준다. 참고로 말하자면 이는 마지막에 위치해야한다.
#특이 바로 밑의 이 부분은 형태소 태그가 이 리스트 안의 것과 일치하는 경우 단순 삭제를 하는 것이고
lis_beta_ef = ['EP+EP+EF', 'EP+EF', 'EF', 'UNKNOWN']

#이 부분 같은 경우는 마지막에 오는 것을 처리하는 것으로 단독으로만 들어가는게 좋겠지?
lis_tag_last = ['EF', 'UNKNOWN']

lis_end = [
    
    'ㅂㄴㅣㄷㅏ', 'ㅅㅣㄷㅏ', 'ㄷㅏㅂㄴㅣㄷㅏ',
    'ㅅㅔㅇㅛ', 'ㄷㅔㅇㅛ', 'ㅇㅔㅇㅛ', 'ㅇㅖㅇㅛ', 'ㄴㅏㅇㅛ', 'ㅇㅡㄹㄲㅏㅇㅛ', 'ㅇㅣㄹㄲㅏㅇㅛ', 'ㄹㄲㅏㅇㅛ', 'ㅇㅡㄴㄱㅏㅇㅛ', 'ㅇㅣㄴㄱㅏㅇㅛ','ㅇㅛ',
    'ㅈㅛ',
    'ㅅㅣㅂㅅㅣㅇㅗ', 'ㅅㅣㅇㅗ', 'ㅇㅗ',
    'ㅂㄴㅣㄲㅏ', 'ㅅㅡㅂㄴㅣㄷㅏ',  
    
]

lis_end_2low = [
    
    'ㄷㅓㄹㅏ','ㄴㄷㅏ', 'ㅆㄷㅏ', 'ㄹㅗㄷㅏ', 'ㄷㅏ', 'ㄱㅔ', 
    'ㄹㅏ',
    'ㅇㅑ', 
    'ㄴㅣㄲㅏ', 
    'ㄲㅏ', 
    'ㄴㅣ', 
    'ㅇㅏ', 
    
]

# lis_end_2low = [
    
#     'ㄷㅓㄹㅏ','ㄴㄷㅏ', 'ㅆㄷㅏ', 'ㄷㅏ', 
#     'ㄹㅏ',
#     'ㅇㅑ', 
    
# ]

P_LIST = ['.', '?', '!', '\'', '\"']

SV_LIST = ['\'', '\"', ':', ';']

lis_plus = [
    
    'EP', 'VCP', 
    
]

In [180]:
mec = Mecab()

In [181]:
def detect_h(input, lis_end_h, lis_end_l):
    for i in lis_end_h:
        if len(input)>=len(i):
            if input[-len(i):]==i:
                return 1
            
    for i in lis_end_l:
        if len(input)>=len(i):
            if input[-len(i):]==i:
                return 0
            
    return -1

def unite(input, dict):
    for i in dict:
        input = re.sub(i[0],i[1],input)
    return input
    
## 자모 단위로 문장을 나누고 합칠 때 쓰는 class ##
class Jamodealer:
    jamo = []
    pp = ''
    #각 단어들을 받아와서 자모단위로 나눈다.
    def __init__(self,lis_word):
    
        self.jamo = []
        for i in lis_word:
            self.jamo.append(split_syllables(i))
    
    ##사전에서 변환된 자모단위로 분리된 문장을 합칠 때 쓰는 함수이다.     
    def make_one(self):
        #list 형태로 저장된 자모들의 집합을 하나의 string pp에 저장한다. 
        self.pp = ''
        for i in self.jamo:
             self.pp= self.pp+i
        ##종성과 종성을 합쳐야 하는 경우가 있다면 합친다.        
        self.pp = unite(self.pp, con_dict)
        
        #자모 단위의 string에서 자모 단위로 사전을 만들고 거기에 index를 부여한다.        
        chars = list(set(self.pp))
        char_to_ix = { ch:i for i,ch in enumerate(chars) }
        ix_to_char = { i:ch for i,ch in enumerate(chars) }
        
        #자모 단위로 분리되었던 문장을 다시 하나로 합친다.
        jamo_numbers = [char_to_ix[x] for x in self.pp]
        restored_jamo = ''.join([ix_to_char[x] for x in jamo_numbers])
        #합쳐진 문장을 return 한다.
        restored_text = join_jamos(restored_jamo)
        return restored_text

def to2lists(input):
    lis_word = []
    lis_tag = []
    #data = han.pos(input,ntags=22,flatten=True, join=False)
    data = mec.pos(input)
    for i in data:
        lis_word.append(i[0])
        lis_tag.append(i[1])
    return lis_word, lis_tag

def to2lists_khaiii(input):
    lis_word = []
    lis_tag = []
    analyzed = api.analyze(input)  
    for data in analyzed:
        for morph in data.morphs:
            lis_word.append(morph.lex)
            lis_tag.append(morph.tag)
    return lis_word, lis_tag


def rememberSpace(lis, input):
    
    rlis = []
    
    for i in range(len(lis)):
        if lis[i]==input:
            rlis.append(i)
            
    for i in range(len(rlis)):
        rlis[i] = rlis[i]-i      
    return rlis

def convertSpace(lis_space,lis_lis):
    
    rlis = []
    k=0
    for i in range(len(lis_lis)):
        
        if k in lis_space:
            rlis.append(i)
            
        k = k+len(lis_lis[i])
        
    #print(rlis)  
    return rlis

def union(lis, lis_lis):
    
    k = 0
    for i in lis:
        lis_lis.insert(i+k,' ')
        k = k+1

def union_t_03(lis_tag):
    
    for i in range(1, len(lis_tag)):
        if lis_tag[i-1] ==' ' or lis_tag[i]==' ':
            lis_tag[i] = lis_tag[i]
        else:
            lis_tag[i] = '/'+lis_tag[i]
            
def union_w_03(lis_w, lis_tag):
    
    for i in range(1, len(lis_w)):
        if lis_tag[i]==' SF':
            lis_w[i] = ' '+lis_w[i+1]

In [182]:
detect_h('누구세요.', lis_end, lis_end_2low)

-1

In [183]:
def prepro(input):
    lis_w, lis_t = to2lists(input)

    space_list = rememberSpace(input,' ')
    space_location = convertSpace(space_list, lis_w)
    union(space_location, lis_w)
    union(space_location, lis_t)
    union_t_03(lis_t)
    union_w_03(lis_w, lis_t)
    
    str_w = ''
    str_t = ''
    for i in range(len(lis_w)):
        str_w = str_w + lis_w[i]
        str_t = str_t + lis_t[i]
    
    data_w = str_w.split(' ')
    data_t = str_t.split(' ')
    
    lis_word, lis_tag = to2lists(input)
    
    lis_ind = []
    t_ind = 0
    jam1 = Jamodealer(lis_word)
    jam2 = Jamodealer(data_w)
    for i in range(len(data_w)):
        element = []
        leng = len(data_t[i].split('/'))
        res = jam2.jamo[i]
        ind = 0
        lenlen = 0
        #element.append(0)
        for j in range(leng):
            element.append(ind)
            ind = ind + len(jam1.jamo[t_ind])
            res = res[len(jam1.jamo[t_ind]):]
            
            lenlen = len(jam1.jamo[t_ind])+lenlen
            t_ind = t_ind+1

        element.append(len(jam2.jamo[i]))
        lis_ind.append(element)
        
    return data_w, data_t, lis_ind

In [184]:
def prepro_khaiii(input):
    lis_w, lis_t = to2lists_khaiii(input)

    space_list = rememberSpace(input,' ')
    space_location = convertSpace(space_list, lis_w)
    union(space_location, lis_w)
    union(space_location, lis_t)
    union_t_03(lis_t)
    union_w_03(lis_w, lis_t)
    
    str_w = ''
    str_t = ''
    for i in range(len(lis_w)):
        str_w = str_w + lis_w[i]
        str_t = str_t + lis_t[i]
    
    data_w = str_w.split(' ')
    data_t = str_t.split(' ')
    
    lis_word, lis_tag = to2lists(input)
    
    lis_ind = []
    t_ind = 0
    jam1 = Jamodealer(lis_word)
    jam2 = Jamodealer(data_w)
    for i in range(len(data_w)):
        element = []
        leng = len(data_t[i].split('/'))
        res = jam2.jamo[i]
        ind = 0
        lenlen = 0
        #element.append(0)
        for j in range(leng):
            element.append(ind)
            ind = ind + len(jam1.jamo[t_ind])
            res = res[len(jam1.jamo[t_ind]):]
            
            lenlen = len(jam1.jamo[t_ind])+lenlen
            t_ind = t_ind+1

        element.append(len(jam2.jamo[i]))
        lis_ind.append(element)
        
    return data_w, data_t, lis_ind

In [185]:
def prepro_beta_06(input, lis_ef, tag_last, lis_w_last, lis_w_last_not):
    data_w, data_t, lis_ind = prepro(input)
    
    last_words = []

    data_w_jamo = []

    data_t_after = []
    
    lis_target_ind = []
    
    for i in data_w:
        jam_ele = Jamodealer(i)
        ele = ''
        for j in jam_ele.jamo:
            ele = ele+j
        data_w_jamo.append(ele)
    
    for i in range(len(data_t)):
        #if i<len(data_t)-1:
        if i<len(data_t):
            lis_res = []
            for ind in range(len(lis_ind[i])-1):
                lis_res.append(data_w_jamo[i][lis_ind[i][ind]:lis_ind[i][ind+1]])

        
        if 'EF/SF' in data_t[i] or 'EF/SV' in data_t[i] or 'UNKNOWN/SF' in data_t[i] or 'UNKNOWN/SV' in data_t[i]:# and 'EC/SF' not in data_t[i]:
            if 'EF/SF' in data_t[i] or 'UNKNOWN' in data_t[i]:
                elements = data_t[i].split('/')
                flag = 0
                
                for j in range(len(elements)):
                
                    flag_end = detect_h(lis_res[j], lis_w_last,  lis_w_last_not)
                    if elements[j] in lis_ef and flag_end==1: #and j == len(elements)-1:
                        elements[j] = 'NULL'
                    
                        last_words.append(data_w_jamo[i][lis_ind[i][j]:lis_ind[i][j+1]])
                        lis_res[j]=''
                    
                        lis_target_ind.append(i)
                    
#                         elements_post = '/'.join(elements)
#                         data_t_after.append(elements_post)
                        
                    elif 'EF' in elements[j] and flag_end==1:# + EF를 처리하는 부분이므로 + EF 만을 마지막에서 처리한다.
                        #print('wwww')
                        for jam in lis_w_last:
                            if len(lis_res[j])>=len(jam):
                                
                                res_out_punc = lis_res[:lis_ind[i][-2]][j]

                                if res_out_punc[-len(jam):]==jam:
                                
                                    lis_target_ind.append(i)
                                
                                    last_words.append(jam)
                                    lis_res[j] = lis_res[j].replace(jam, '', 1)

                                    for k in tag_last:

                                        if k in elements[j]:
                                        
                                            if '+' in elements[j]:
                                            
                                                ind = elements[j].index('+'+k)
                                                elements[j] = elements[j][:ind]
                                    break
                                
                                elif lis_w_last.index(jam)==len(lis_w_last)-1:#new
                                    for k in tag_last:

                                        if k in elements[j]:
                                        
                                            if '+' in elements[j]:
                                            
                                                ind = elements[j].index('+'+k)
                                                elements[j] = elements[j][:ind]
                                            
                                    lis_target_ind.append(i)
                                    last_words.append('')
                                    break

                
                elements_post = '/'.join(elements)
                data_t_after.append(elements_post)
                
                data_w_jamo[i] = ''.join(lis_res)

                #elements_post = '/'.join(elements)
                #data_t_after.append(elements_post)
                    
            #######################################
            
            #data_t_after.append(data_t[i])
            
        elif 'EC/SF' in data_t[i]:
            #print('ee')
            elements = data_t[i].split('/')
            
            flag = 0
            for j in range(len(elements)):
                
                flag_end = -1
                if elements[j]=='EC':
                    flag_end = detect_h(lis_res[j], lis_w_last_not, lis_w_last)
                        
                #print(elements[j])
                #print(flag_end)
                if flag_end==0 and i not in lis_target_ind:
                    lis_target_ind.append(i)
                    last_words.append('')
            #print(data_t[i])
            data_t_after.append(data_t[i])
            
        ####### #######
            
        else:
            data_t_after.append(data_t[i])
            
        
        #################################################################
        
#         if 'EP/EF' in data_t[i]:
#             #print('see')
#             elements = data_t[i].split('/')
            
#             flag = 0
#             for j in range(len(elements)):
                
#                 flag_end = -1
#                 if elements[j]=='EP':
#                     if lis_res[j]=='ㅅㅣ':
#                         flag_end =0
                        
#                 #print(elements[j])
#                 #print(flag_end)
#                 if flag_end==0 and i in lis_target_ind:
                    
#                     last_words[-1] = 'ㅅㅣ'+last_words[-1]
#                     lis_res[j]=''
#                     #print(lis_res)
#             #print(data_t[i])
#             data_t_after.append(data_t[i])
#             data_w_jamo[i] = ''.join(lis_res)
            
        lis_normal = []
    
        for i in data_w_jamo:
            jam_n = Jamodealer(i)
            lis_normal.append(jam_n.make_one())
    
#         lis_target_ind = []

#         word_test_pre, tag_test_pre, ind_test_pre = prepro(input)
    
    
#         for i in range(len(lis_normal)):
#             if lis_normal[i]!=data_w[i]:
#                 lis_target_ind.append(i)

        
        
        
    return data_w, data_t, lis_ind, data_w_jamo, data_t_after, last_words, lis_target_ind
                        
                    
   

In [205]:
def prepro_beta_khaiii(input, lis_ef, tag_last, lis_w_last, lis_w_last_not):
    data_w, data_t, lis_ind = prepro_khaiii(input)
    
    last_words = []

    data_w_jamo = []

    data_t_after = []
    
    lis_target_ind = []
    
    for i in data_w:
        jam_ele = Jamodealer(i)
        ele = ''
        for j in jam_ele.jamo:
            ele = ele+j
        data_w_jamo.append(ele)
    
    for i in range(len(data_t)):
        #if i<len(data_t)-1:
        if i<len(data_t):
            lis_res = []
            for ind in range(len(lis_ind[i])-1):
                lis_res.append(data_w_jamo[i][lis_ind[i][ind]:lis_ind[i][ind+1]])

        
        if 'EF/SF' in data_t[i] or 'EF/SV' in data_t[i] or 'UNKNOWN/SF' in data_t[i] or 'UNKNOWN/SV' in data_t[i]:# and 'EC/SF' not in data_t[i]:
            if 'EF/SF' in data_t[i] or 'UNKNOWN' in data_t[i]:
                elements = data_t[i].split('/')
                flag = 0
                
                for j in range(len(elements)):
                
                    flag_end = detect_h(lis_res[j], lis_w_last,  lis_w_last_not)
                    if elements[j] in lis_ef and flag_end==1: #and j == len(elements)-1:
                        elements[j] = 'NULL'
                    
                        last_words.append(data_w_jamo[i][lis_ind[i][j]:lis_ind[i][j+1]])
                        lis_res[j]=''
                    
                        lis_target_ind.append(i)
                    
#                         elements_post = '/'.join(elements)
#                         data_t_after.append(elements_post)
                        
                    elif 'EF' in elements[j] and flag_end==1:# + EF를 처리하는 부분이므로 + EF 만을 마지막에서 처리한다.
                        #print('wwww')
                        for jam in lis_w_last:
                            if len(lis_res[j])>=len(jam):
                                
                                res_out_punc = lis_res[:lis_ind[i][-2]][j]

                                if res_out_punc[-len(jam):]==jam:
                                
                                    lis_target_ind.append(i)
                                
                                    last_words.append(jam)
                                    lis_res[j] = lis_res[j].replace(jam, '', 1)

                                    for k in tag_last:

                                        if k in elements[j]:
                                        
                                            if '+' in elements[j]:
                                            
                                                ind = elements[j].index('+'+k)
                                                elements[j] = elements[j][:ind]
                                    break
                                
                                elif lis_w_last.index(jam)==len(lis_w_last)-1:#new
                                    for k in tag_last:

                                        if k in elements[j]:
                                        
                                            if '+' in elements[j]:
                                            
                                                ind = elements[j].index('+'+k)
                                                elements[j] = elements[j][:ind]
                                            
                                    lis_target_ind.append(i)
                                    last_words.append('')
                                    break

                
                elements_post = '/'.join(elements)
                data_t_after.append(elements_post)
                
                data_w_jamo[i] = ''.join(lis_res)

                #elements_post = '/'.join(elements)
                #data_t_after.append(elements_post)
                    
            #######################################
            
            #data_t_after.append(data_t[i])
            
        elif 'EC/SF' in data_t[i]:
            #print('ee')
            elements = data_t[i].split('/')
            
            flag = 0
            for j in range(len(elements)):
                
                flag_end = -1
                if elements[j]=='EC':
                    flag_end = detect_h(lis_res[j], lis_w_last_not, lis_w_last)
                        
                #print(elements[j])
                #print(flag_end)
                if flag_end==0 and i not in lis_target_ind:
                    lis_target_ind.append(i)
                    last_words.append('')
            #print(data_t[i])
            data_t_after.append(data_t[i])
            
        ####### #######
            
        else:
            data_t_after.append(data_t[i])
            
        
        #################################################################
        
#         if 'EP/EF' in data_t[i]:
#             #print('see')
#             elements = data_t[i].split('/')
            
#             flag = 0
#             for j in range(len(elements)):
                
#                 flag_end = -1
#                 if elements[j]=='EP':
#                     if lis_res[j]=='ㅅㅣ':
#                         flag_end =0
                        
#                 #print(elements[j])
#                 #print(flag_end)
#                 if flag_end==0 and i in lis_target_ind:
                    
#                     last_words[-1] = 'ㅅㅣ'+last_words[-1]
#                     lis_res[j]=''
#                     #print(lis_res)
#             #print(data_t[i])
#             data_t_after.append(data_t[i])
#             data_w_jamo[i] = ''.join(lis_res)
            
        lis_normal = []
    
        for i in data_w_jamo:
            jam_n = Jamodealer(i)
            lis_normal.append(jam_n.make_one())
    
#         lis_target_ind = []

#         word_test_pre, tag_test_pre, ind_test_pre = prepro(input)
    
    
#         for i in range(len(lis_normal)):
#             if lis_normal[i]!=data_w[i]:
#                 lis_target_ind.append(i)

        
        
        
    return data_w, data_t, lis_ind, data_w_jamo, data_t_after, last_words, lis_target_ind
                        
                    
   

In [222]:
print(prepro_beta_06('노세요.', lis_beta_ef, lis_tag_last, lis_end, lis_end_2low))

(['노세요.'], ['VV+EP+EF/SF'], [[0, 6, 7]], ['ㄴㅗ.'], ['VV+EP/SF'], ['ㅅㅔㅇㅛ'], [0])


In [223]:
ori_w, ori_t, ind, converted_w, converted_t, last_ef,target_ind  = prepro_beta_06('먹으십니다.', lis_beta_ef, lis_tag_last, lis_end, lis_end_2low)

In [224]:
ori_w, converted_w,converted_t, last_ef

(['먹으십니다.'], ['ㅁㅓㄱ.'], ['VV/NULL/SF'], ['ㅇㅡㅅㅣㅂㄴㅣㄷㅏ'])

In [189]:
######높임말 -> 반말
#현재 만들어진 것은 EF만 잘라낼 것이다. 
#원래 ef사전에 mapping 되는 것을 찾아낸다.
# EF_1 = {
#     ['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']:['ㄷ','ㅏ'],
#     ['ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']:['ㄷ','ㅏ']
# }
#python dictionary로 접근
#종결어미 처리
EF = {
    ###하십시오체###
    #평서문
    #'ㅂㄴㅣㄷㅏ': 'ㄷㅏ',
    'ㅂㄴㅣㄷㅏ': 'special3',
    'ㅅㅡㅂㄴㅣㄷㅏ':'special2',
    'ㅇㅗㄹㅅㅣㄷㅏ':'ㄷㅏ', #**
    'ㅂㅈㅣㅇㅛ':'지', #**
    'ㅅㅣㅂㄴㅣㄷㅏ':'special1',
    #의문문
    'ㅅㅡㅂㄴㅣㄲㅏ':'ㄴㅣ',
    'ㅅㅣㅂㄴㅣㄲㅏ':'special1', #EP+EF
    #명령법
    'ㅇㅡㅅㅔㅇㅛ': 'special0',
    'ㅅㅔㅇㅛ':'special1',
    'ㅅㅣㅂㅅㅣㅇㅗ':'ㅅㅣㅇㅗ',
    #청유법
    
    #명령인용법
    
    ###하오체###
    
    ###해요체###
    'ㅇㅓㅇㅛ':'ㅇㅓ',
    'ㅇㅏㅇㅛ':'ㅇㅏ',
    
}
#'ㄹ'규칙 활용 -> ㄹ 규칙 활용이 일어나는 동사들을 최대한 모아두고, 만약 하나의 단어에 여러 의미가 담긴다면?
EF_SaeYo_R= {
    'ㄱㅜ':'ㄹ',
    'ㄴㅗ':'ㄹ',
    'ㄴㅏ':'ㄹ',
    'ㄷㅗ':'ㄹ',
    'ㄷㅡ':'ㄹ',
    'ㄷㅏ':'ㄹ',
    'ㄷㅜ':'ㄹ',
    'ㅂㅜ':'ㄹ',
    'ㄲㅗ':'ㄹ',
    'ㅁㅣ':'ㄹ',
    'ㅁㅜ':'ㄹ',
    #'ㅂㅗㅍㅜ':'ㄹ', #error predicate 수정
    'ㅂㅜ':'ㄹ',
    'ㅅㅡ':'ㄹ',
    'ㄸㅓ':'ㄹ',   
}

####EXAMPLE####
#tag = 'ㅅㅔㅇㅛ'
# sentence = ['ㄱㅡㄴㅡㄴ', 'ㅅㅏㄴㅇㅡㄹ', 'ㅇㅗㄹㅡ']
# tagList = ['NP/JX', 'NNG/JKO', 'VV/']

# tag2 = 'ㅅㅔㅇㅛ'
# sentence2 = ['ㄱㅡㄴㅡㄴ', 'ㅂㅐㄹㄱㅗㅍㅡ']
# tagList2 = ['NP/JX', 'VA/']
####EXAMPLE####
def treatSF(stc, ex):
    ind_point = -1
    point = ''          
    for i in range(len(stc)):
        if stc[i] in P_LIST:
            point = stc[i]
            break
            
    if point in P_LIST:
        ind_point = stc.index(point)
    
    r_word = ''
    r_pun = ''
    
    if ind_point!=-1:
        r_word = stc[:ind_point]
        r_pun = stc[ind_point:]
    else:
        r_word = stc
    return r_word+ex+r_pun

## 'ㅏ', 'ㅗ' 처리
def convertSpecialCase_AhOh(sentence):
    #print(sentence[-1][-2:])
    #print(sentence[-1][-4:])
    ## 수정할 필요 있음!
    if sentence[-3:-1] == 'ㅍㅡ' or sentence[-3:-1] == 'ㅃㅡ':
        if sentence[-5:-3].find('ㅏ') !=-1 or sentence[-5:-3].find('ㅗ')!=-1:
            return 'ㅏ'
        else:
            return 'ㅓ'
    elif sentence[-3:].find('ㅏ') !=-1 or sentence[-3:].find('ㅗ') !=-1:
        return 'ㅇㅏ'
    else:
        return 'ㅇㅓ'

#'ㅇㅗㄹㅡ'
def convertSpecialCase_SaeYo(sentence, tag):
    result = ''
    end_EF=''
    final=''
    predicate = sentence[-3:-1]
    stem = sentence[:-3]
    print(predicate, 'ssdads')
    
    #만약 VCP가 있다면 '야'를 붙이고 return 한다.
    
    # 'ㄹ'규칙 활용
    if predicate in EF_SaeYo_R:
        result= EF_SaeYo_R[predicate]
        #sentence[-1] += result
        ##'아' 또는 '어' 로 처리
        end_EF = convertSpecialCase_AhOh(sentence)
        #sentence.append(end_EF)
        final = result +end_EF
        #print(final)
    #'르' 불규칙 활용
    elif predicate =='ㄹㅡ':
        # 용언 종성에 ㄹ이 있다면
        #sentence[-1] = sentence[-1].replace('ㄹㅡ','')
        predicate = predicate.replace('ㄹㅡ','')
        sentence = stem+predicate
        end_EF = convertSpecialCase_AhOh(sentence)
        end_EF = end_EF[-1]
        #print(end_EF, sentence, 'sss')
        if sentence[-1].find('ㄹ') != -1:
            final = 'ㄹ'+end_EF
        else:
            final  = 'ㄹㄹ'+end_EF
        
    #'우' 불규칙 활용
    # '푸'를 제외한 다른 'ㅜ'는 'ㅓ'와 결합
    elif predicate.find('ㅍㅜ') !=-1:
        #sentence[-1] = sentence[-1].replace('ㅜ','ㅓ')
        predicate = predicate.replace('ㅜ','')
        sentence = stem+predicate
        final = 'ㅓ'
        #final = sentence[-1][-1:]
    #'오' 불규칙 활용(고려하지 않을 수 있음)
    #'하' 불규칙 활용
    elif predicate.find('ㅎㅏ') !=-1:
        #sentence[-1] = sentence[-1].replace('ㅏ','ㅐ')
        predicate = predicate.replace('ㅏ','')
        sentence = stem + predicate
        final = 'ㅐ'
    #활용이 안되었던 용언 처리
    else:
        if predicate.find('ㅡ') !=-1:
            end_EF = convertSpecialCase_AhOh(sentence)
            #sentence[-1] = sentence[-1].replace('ㅡ',end_EF)
            predicate = predicate.replace('ㅡ','')
            sentence = stem + predicate
            final = end_EF[-1]
        ## 수정할 필요 있음!!
        elif predicate.find('ㅗ') !=-1:
            #sentence[-1] = sentence[-1].replace('ㅗ','ㅘ')
            predicate = predicate.replace('ㅗ','')
            sentence = stem+predicate
            final = 'ㅘ'
        elif predicate.find('ㅜ') !=-1:
            #sentence[-1] = sentence[-1].replace('ㅜ','ㅝ')
            predicate = predicate.replace('ㅜ','') 
            sentence = stem + predicate
            final = 'ㅝ'
        elif predicate.find('ㅣ') !=-1:
            #sentence[-1] = sentence[-1].replace('ㅣ','ㅕ')
#             if predicate == 'ㅇㅣ':
#                 sentence = sentence.replace('ㅇㅣ','')
            predicate = predicate.replace('ㅣ','')
            sentence = stem+predicate
            final = 'ㅕ'
        else:
            final = convertSpecialCase_AhOh(sentence)
            #sentence = sentence.append(final)
            if  predicate.find('ㅏ') !=-1:
                final = ''
            elif  predicate.find('ㅓ') !=-1:
                final = ''
#                 return final, sentence
#             return final, sentence
#     return final, sentence
    converted = treatSF(sentence, final)
    return converted
    
def convertSpecialCase_Da(sentence, ending, taglist):
    print(sentence, ending, taglist)
    final =''
    if taglist.find('EP/NULL') !=-1 or taglist.find('VA/NULL') !=-1 or taglist.find('VX/NULL') !=-1:
        #print('다로 변경할 것')
        final = 'ㄷㅏ'
    elif taglist.find('VV/NULL') !=-1:
        #print('현재형 동사가 왔으므로, 는다로 변경할 것')
        final = 'ㄴㅡㄴㄷㅏ'
    converted = treatSF(sentence, final)
    return converted
def convertSpecialCase_Nida(sentence, ending, taglist):
    print(sentence, ending, taglist)
    #시가 있으면 ㄴ다를 붙인다. 
    #형용사, 서술격 조사일 경우 convertSpecialCase_SaeYo를 통해 변경한 다음, 아/어를 제거 후 다를 붙이고
    #동사일 경우 ㄴ다를 붙여서 해결한다. 
##if Verb & adjective
## '-시' 등 선어말 처리
## 습니다는 동사면 '는다', 그외에는 '다'로 간다
def treat_ending(sentence, ending, taglist):
    re_value =''
    temp =''
    
    for key in EF:
        if ending == key:
            re_value = EF[key]
            #나요, 으세요(으세요, 세요 모두 함수에서 커버 가능), 습니까 case
            if re_value == 'special0':
                #시처리하기 
                temp = convertSpecialCase_AhOh(sentence)
                re_value = treatSF(sentence, temp)
                #ㅗ,ㅜ 이면 ㅘ, ㅝ로 결합할 것
                #print(re_value)
            #-세요,십니다, 십니까
            elif re_value == 'special1':
                #시처리하기
                re_value = convertSpecialCase_SaeYo(sentence, ending)
            elif re_value == 'special2':
                re_value = convertSpecialCase_Da(sentence, ending, taglist)
            elif re_value == 'special3':
                #시처리 안함 '시' 보존
                re_value = convertSpecialCase_Nida(sentence, ending, taglist)
            else:
                re_value = treatSF(sentence,re_value)
    # print(sentence)
    return re_value
#def treat_others(sentence, ending):
    
# 동사&형용사인지 아닌지를 먼저 판단하고 동사 및 형용사이면 treatVerb_Adjective 함수로 보내고 아니면 treat_others로 보낸다. 


# punc = cont_w[-1][-1]
# cont_w[-1] = cont_w[-1][:-1]
# print(cont_w)
# print(ending)
# end = ''.join(ending)
# re_value = treatVerb_Adjective(cont_w, end)
# print(re_value)
# cont_w[-1] += re_value
# #print(sentence)

# res_space = []
# for i in range(len(cont_w)):
#     res_space.append(cont_w[i])
#     if i==len(cont_w)-1:
#         break
#     res_space.append(' ')
# jam_q = Jamodealer(res_space)

# result = jam_q.make_one()
# print(result + punc)
#VX는 따로 처리해야 함

# # key_list[0]
# def ChangeMorphemeToLow(tag, sentence):
#     #for문으로 ef list 탐색 후 일치한다면 변환 후 return 
#     #key_list = EF_1.keys()
#     for key in EF:
#         if tag == key:
#             if EF
#             tag = EF_1[key]
#     print(tag)
#     return tag

In [190]:
sen=converted_w[target_ind[0]]

In [191]:
converted_t[target_ind[0]]

'VV/NULL/SF'

In [192]:
end = treatVerb_Adjective(converted_w[target_ind[0]], last_ef[0], converted_t[target_ind[0]])
# treatSF(s,end)
print(end)

In [193]:
converted_w, converted_t, last_ef,target_ind

(['ㅁㅓㄱ.'], ['VV/NULL/SF'], ['ㅇㅡㅅㅣㅂㄴㅣㄷㅏ'], [0])

In [194]:
converted_w[-1][-3:-1]

'ㅓㄱ'

In [195]:
s = converted_w[target_ind[0]]
s[-5:-3]

'ㅁ'

In [196]:
s1 = 'ㅇㅗ.'
s1[-3:]

'ㅇㅗ.'

In [197]:
def make_end_l(self, stc, tag, ex):
    #ex는 종결어미, stc는 종결어미를 제외한 종결부분 어간, tag는 종결어미를 뺀 tag
    print(stc, tag, ex)
    if 'ㄷㅏ'== ex[-2:]:
        return convert_da_low(stc, tag, ex)   
    else:
        return convert_an_low(stc, tag, ex)
    

In [198]:
for i in range(len(lis_target_ind)):
    new_end = self.make_end_h(lis_w[lis_target_ind[i]], lis_t[lis_target_ind[i]], off_word[i])
    jam.jamo[lis_target_ind[i]] = new_end

NameError: name 'lis_target_ind' is not defined

In [201]:
from khaiii import KhaiiiApi
khai = KhaiiiApi()
for word in khai.analyze('안녕, 세상.'):
    print(word)

안녕,	안녕/IC + ,/SP
세상.	세상/NNG + ./SF


In [214]:
def to2lists_khaiii(input):
    lis_word = []
    lis_tag = []
    analyzed = khai.analyze(input)  
    for data in analyzed:
        for morph in data.morphs:
            lis_word.append(morph.lex)
            lis_tag.append(morph.tag)
    return lis_word, lis_tag

In [219]:
txt = '밥 먹으세요.'
lis_word, lis_tag = to2lists_khaiii(txt)
lis_word1, lis_tag1 = to2lists(txt)

In [220]:
lis_word, lis_tag

(['밥', '먹', '으시', '어요', '.'], ['NNG', 'VV', 'EP', 'EF', 'SF'])

In [221]:
lis_word1, lis_tag1

(['밥', '먹', '으세요', '.'], ['NNG', 'VV', 'EF', 'SF'])

In [155]:
stn = 'ㅎㅣㅁㄷㅡㅇㅡㅅㅣ.'
taglist = 'VA/EP/NULL/SF'
last_ef = 'ㄴㅏㅇㅛ'
def delete_EP_si(stn, taglist):
    print(stn, taglist)
    

In [170]:
sisi1 = stn[-3:]
sisi2 = stn[-5:]
taglist[-11:-2]
# if sisi in stn:
#     print(sisi)

'/EP/NULL/'

In [164]:
sisi2

'ㅇㅡㅅㅣ.'